# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 19 2022 10:30AM,242201,20,ALUR740272551,Alumier Labs Inc.,Released
1,Jul 19 2022 10:30AM,242201,20,ALU0007341,Alumier Labs Inc.,Released
2,Jul 19 2022 10:30AM,242201,20,ALUR753314814,Alumier Labs Inc.,Released
3,Jul 19 2022 10:30AM,242201,20,ALU0007342,Alumier Labs Inc.,Released
4,Jul 19 2022 10:30AM,242201,20,ALUR782355816,Alumier Labs Inc.,Released
5,Jul 19 2022 10:30AM,242201,20,ALU0007343,Alumier Labs Inc.,Released
6,Jul 19 2022 10:30AM,242201,20,ALUR495961445,Alumier Labs Inc.,Released
7,Jul 19 2022 10:30AM,242201,20,ALUR535966107,Alumier Labs Inc.,Released
8,Jul 19 2022 10:30AM,242201,20,ALUR645418770,Alumier Labs Inc.,Released
9,Jul 19 2022 10:30AM,242201,20,ALUR948701773,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
57,242196,Released,1
58,242198,Released,1
59,242199,Released,1
60,242200,Released,1
61,242201,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242196,NaN,NaN,1.0
242198,NaN,NaN,1.0
242199,NaN,NaN,1.0
242200,NaN,NaN,1.0
242201,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4.0,5.0,3.0
242008,0.0,0.0,1.0
242020,17.0,0.0,2.0
242067,0.0,0.0,1.0
242075,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4,5,3
242008,0,0,1
242020,17,0,2
242067,0,0,1
242075,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,5,3
1,242008,0,0,1
2,242020,17,0,2
3,242067,0,0,1
4,242075,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,5,3
1,242008,,,1
2,242020,17,,2
3,242067,,,1
4,242075,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 19 2022 10:30AM,242201,20,Alumier Labs Inc.
14,Jul 19 2022 10:25AM,242200,19,"NAPP Technologies, LLC"
15,Jul 19 2022 10:24AM,242199,10,"ClearSpec, LLC"
16,Jul 19 2022 10:12AM,242198,10,"ClearSpec, LLC"
17,Jul 19 2022 10:09AM,242196,15,"Person & Covey, Inc."
18,Jul 19 2022 9:57AM,242152,15,"Carwin Pharmaceutical Associates, LLC"
20,Jul 19 2022 9:56AM,242151,15,"Virtus Pharmaceuticals, LLC"
33,Jul 19 2022 9:51AM,242150,15,"Person & Covey, Inc."
57,Jul 19 2022 9:51AM,242148,15,"Brookfield Pharmaceuticals, LLC"
93,Jul 19 2022 9:46AM,242143,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 19 2022 10:30AM,242201,20,Alumier Labs Inc.,,,14
1,Jul 19 2022 10:25AM,242200,19,"NAPP Technologies, LLC",,,1
2,Jul 19 2022 10:24AM,242199,10,"ClearSpec, LLC",,,1
3,Jul 19 2022 10:12AM,242198,10,"ClearSpec, LLC",,,1
4,Jul 19 2022 10:09AM,242196,15,"Person & Covey, Inc.",,,1
5,Jul 19 2022 9:57AM,242152,15,"Carwin Pharmaceutical Associates, LLC",,,2
6,Jul 19 2022 9:56AM,242151,15,"Virtus Pharmaceuticals, LLC",,,13
7,Jul 19 2022 9:51AM,242150,15,"Person & Covey, Inc.",,,24
8,Jul 19 2022 9:51AM,242148,15,"Brookfield Pharmaceuticals, LLC",,6,30
9,Jul 19 2022 9:46AM,242143,15,"Alliance Pharma, Inc.",,,20


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 10:30AM,242201,20,Alumier Labs Inc.,14,,
1,Jul 19 2022 10:25AM,242200,19,"NAPP Technologies, LLC",1,,
2,Jul 19 2022 10:24AM,242199,10,"ClearSpec, LLC",1,,
3,Jul 19 2022 10:12AM,242198,10,"ClearSpec, LLC",1,,
4,Jul 19 2022 10:09AM,242196,15,"Person & Covey, Inc.",1,,
5,Jul 19 2022 9:57AM,242152,15,"Carwin Pharmaceutical Associates, LLC",2,,
6,Jul 19 2022 9:56AM,242151,15,"Virtus Pharmaceuticals, LLC",13,,
7,Jul 19 2022 9:51AM,242150,15,"Person & Covey, Inc.",24,,
8,Jul 19 2022 9:51AM,242148,15,"Brookfield Pharmaceuticals, LLC",30,6,
9,Jul 19 2022 9:46AM,242143,15,"Alliance Pharma, Inc.",20,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 10:30AM,242201,20,Alumier Labs Inc.,14,,
1,Jul 19 2022 10:25AM,242200,19,"NAPP Technologies, LLC",1,,
2,Jul 19 2022 10:24AM,242199,10,"ClearSpec, LLC",1,,
3,Jul 19 2022 10:12AM,242198,10,"ClearSpec, LLC",1,,
4,Jul 19 2022 10:09AM,242196,15,"Person & Covey, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 10:30AM,242201,20,Alumier Labs Inc.,14.0,NaN,NaN
1,Jul 19 2022 10:25AM,242200,19,"NAPP Technologies, LLC",1.0,NaN,NaN
2,Jul 19 2022 10:24AM,242199,10,"ClearSpec, LLC",1.0,NaN,NaN
3,Jul 19 2022 10:12AM,242198,10,"ClearSpec, LLC",1.0,NaN,NaN
4,Jul 19 2022 10:09AM,242196,15,"Person & Covey, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 10:30AM,242201,20,Alumier Labs Inc.,14.0,0.0,0.0
1,Jul 19 2022 10:25AM,242200,19,"NAPP Technologies, LLC",1.0,0.0,0.0
2,Jul 19 2022 10:24AM,242199,10,"ClearSpec, LLC",1.0,0.0,0.0
3,Jul 19 2022 10:12AM,242198,10,"ClearSpec, LLC",1.0,0.0,0.0
4,Jul 19 2022 10:09AM,242196,15,"Person & Covey, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3632495,116.0,31.0,17.0
102,242008,1.0,0.0,0.0
12,484372,10.0,1.0,0.0
15,1452940,90.0,6.0,0.0
18,726535,3.0,0.0,0.0
19,968658,7.0,7.0,17.0
20,1694698,146.0,5.0,4.0
21,3389655,14.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3632495,116.0,31.0,17.0
1,102,242008,1.0,0.0,0.0
2,12,484372,10.0,1.0,0.0
3,15,1452940,90.0,6.0,0.0
4,18,726535,3.0,0.0,0.0
5,19,968658,7.0,7.0,17.0
6,20,1694698,146.0,5.0,4.0
7,21,3389655,14.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,116.0,31.0,17.0
1,102,1.0,0.0,0.0
2,12,10.0,1.0,0.0
3,15,90.0,6.0,0.0
4,18,3.0,0.0,0.0
5,19,7.0,7.0,17.0
6,20,146.0,5.0,4.0
7,21,14.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,116.0
1,102,Released,1.0
2,12,Released,10.0
3,15,Released,90.0
4,18,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,18,19,20,21
Status,,,,,,,,
Completed,17.0,0.0,0.0,0.0,0.0,17.0,4.0,0.0
Executing,31.0,0.0,1.0,6.0,0.0,7.0,5.0,0.0
Released,116.0,1.0,10.0,90.0,3.0,7.0,146.0,14.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,18,19,20,21
0,Completed,17.0,0.0,0.0,0.0,0.0,17.0,4.0,0.0
1,Executing,31.0,0.0,1.0,6.0,0.0,7.0,5.0,0.0
2,Released,116.0,1.0,10.0,90.0,3.0,7.0,146.0,14.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,18,19,20,21
0,Completed,17.0,0.0,0.0,0.0,0.0,17.0,4.0,0.0
1,Executing,31.0,0.0,1.0,6.0,0.0,7.0,5.0,0.0
2,Released,116.0,1.0,10.0,90.0,3.0,7.0,146.0,14.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()